# JAXitude Noise Models
JAXitude provides models for injecting error into 'true' pose data (the actual attitude rate vector at a given time for a spacecraft, for example). Trajectories and orientations generated via a physical simulation are incredibly useful for testing localization, SLAM, and control algorithms.  But in order to test these algorithms, one needs to also simulate measurement error by adding realistic noise to the simulated 'true' data.  For some data, this is trivial.  For example, simulating a measured displacement vector $\mathbf{x}_{\mathcal{R}}^*$ from the true displacement vector $\mathbf{x}_{\mathcal{R}}$ of a spacecraft relative to some intertial frame $\mathcal{R}$ is pretty straightforward:

$\mathbf{x}_{\mathcal{R}}^* = \mathbf{x}_{\mathcal{R}} + \delta\mathbf{x}_{\mathcal{R}}$, where $\delta\mathbf{x}_{\mathcal{R}} \sim \mathcal{N}(0, \Sigma_{\delta\mathbf{x}_{\mathcal{R}}})$.

Many data types used to described dynamical quantities do obey such noise models, though.  Position $\mathbf{x}$, along with other linear kinematic quantities such as velocity $\dot{\mathbf{x}}$, are vectors, meaning any two vectors added together result in another vector (as seen above).  Many dynamical quantities are not vectors, though, and require more careful and nontrivial treatment. JAXitude noise models can help with that.

JAX handles random number generation differently than numpy.  Most important is that you need to feed keys into each JAX function that utilizes a random number generator --- JAXitude requires the same thing!  See the tutorial [Pseudo Random Numbers in JAX](https://jax.readthedocs.io/en/latest/jax-101/05-random-numbers.html) for an introduction to and summary of JAX random number API.  

Unless otherwise noted, rotation errors will be assumed to be Gaussian and small (I.e. $\delta\theta \ll \pi/2$).


In [9]:
from pathlib import Path
import sys
import os
sys.path.append(str(Path(os.getcwd()).parent))

## Attitude Noise Models
To start, let's consider attitudes.  An object's orientation is described by a rotation, itself which can be represented using various techniques including, but not limited to, rotation matrices $[\mathbf{R}]$ (also called directed cosine matrices, or DCMs), quaternions $\beta$, and Rodrigues parameters $q$ or $\sigma$.  Importantly, rotations are elements of the $\text{SO}(3)$ group, meaning that you cannot simply add two rotations to get a new rotation, since $\text{SO}(3)$ is not closed under addition.  Instead, groups are closed under composition.  That means that if we can describe some rotation error, then via composition we can compose the 'true' rotation and the rotation error to get a measured rotation. How the $\text{SO}(3)$ composition operation is calculated depends on one's chosen representation, which in turn will determine how one composes a rotation error with the 'true' rotation to simulate a measured rotation. 


### Quaternion Noise Models
Quaternions $\beta \in \mathcal{H}_1$, where $\mathcal{H}_1$ is the set of quaternions of unit length ($|\beta|=1$), are a valid representation of $\text{SO}(3)$.  They are particularly useful in practice since they have no singularities that otherwise plague other rotation representations (such as Euler angles).  Adding two unit quaternions $\beta_p$ and $\beta_{pp}$ does no yield another quaterion --- instead, quaternion multiplication is used to compose two quaternions:

$\beta = \beta_{p} \otimes \beta_{pp}$.

JAXitude's quaternion composition function `compose_quat(b_p, b_pp)` can be found in the `jaxitude.operations.composition` submodule.

Naively, one might think that we can compose a random 4D vector $\delta\mathbf{v} \sim \mathcal{N}(0, \Sigma_{\mathbf{v}})$, normalize it to get a quaternion error $\delta\beta$, and then compose $\delta\beta$ with the 'true' quaternion $\beta$ to simulate a measured quaternion $\beta^*$:

$\beta^* = \beta \otimes \delta\beta$.

This does not work, unfortunately.  It is incredibly unclear what the covariance $\Sigma_{\mathbf{v}}$ would have to be in order to recover a $\delta\beta$ that describes small rotations, which is what we want for this error model.  We could instead try to Taylor expand the definition of $\beta$ around a zero rotation and then sample said angle form a normal distribution centered at zero, but we run into the issue that such a linear approximation will not yield actual quaternions! Furthermore, we aren't even considering the principal rotation axis yet, which determines the rotation plane in which the rotation error is applied.

To overcome these issues, JAXitude uses the quaternion exponential map '$\exp_{\beta}$' (available via `jaxitude.quaternions.quat_expm(angle, e)`) after calculating the 'true' $\beta$'s principal rotation axis $\mathbf{e}$.  The rotation error $\delta\theta$ is sampled from a normal distribution $\mathcal{N}(0, \sigma_{\delta\theta})$ and used to build the error quaternion $\delta\beta$, from which the measured quaternion $\beta^*$ is calculated via quaternion composition:

$\beta^* = \beta \otimes \exp_{\beta}([0, \delta\theta\mathbf{e}]^T)$.

Quaternion noise functionality is found in the class `jaxitude.operations.noise.QuatNoise`.  Let's explore how to simulated measured quaternions by adding principal rotation angle error using `QuatNoise.addnoise()`.

In [5]:
import jax.numpy as jnp
from jax.random import PRNGKey, split
from jax import vmap

from jaxitude.operations.noise import QuatNoise

key = PRNGKey(1)

# The true simulated quaternion will be a pi rotation along the x-axis.
b_true = jnp.array(
    [[0.],
     [1.],
     [0.],
     [0.]]
)

# The angle error standard deviation.
sigma_dtheta = 0.1  # in radians.

# wrapper function to generate N measured b from b_true.
N = 50
@vmap
def QuatNoise_sample(key: int):
    return QuatNoise.addnoise(key, b_true, sigma_dtheta)

# Split key and get sample of observed bs.
key, subkey = split(key)
bs_obs = QuatNoise_sample(split(subkey, N))

print(bs_obs[0])

[[-0.01591068]
 [ 0.99987346]
 [ 0.        ]
 [ 0.        ]]


There may be a scenario where you want to perturb a 'true' quaternion $\beta$'s principal rotation angle *and* its principal rotation axis.  This requires an axis deflection angle $\delta\phi$.  First, JAXitude will calculate the principal rotation axis $\mathbf{e}$ from $\beta$ like before, but will then calculate a new principal rotation axis $\mathbf{e}^{\prime}(\delta\phi)$ which is a linear combination of $\mathbf{e}$ and a random unit vector $\mathbf{e}_{\perp}$ which is perpendicular to $\mathbf{e}$:

$\mathbf{e}^{\prime}(\delta\phi) = \cos(\delta\phi)\mathbf{e} + \sin(\delta\phi)\mathbf{e}_{\perp}$, where $\delta\phi \sim \mathcal{N}(0, \sigma_{\delta\phi})$.

Finally, the observed quaternion beta $\beta^*$ is calculate as before, but using $\mathbf{e}^{\prime}(\delta\phi)$ instead of $\mathbf{e}$:

$\beta^* = \beta \otimes \exp_{\beta}([0, \delta\theta\mathbf{e}^{\prime}(\delta\phi)]^T)$.

Word of caution: $\delta\phi$ should be small and should be smaller than $\delta\theta$ (more specifically, $\sigma_{\delta\phi}<\sigma_{\delta\theta}$).

To add perturb the rotation angle via $\delta\theta$ and the rotation axis via deflection angle $\delta\phi$, use `QuatNoise.addnoise_perturbaxis()`.

In [7]:
# Define the deflection angle standard deviation.
sigma_dphi = 0.05  # in radians.

# wrapper function to generate N measured b from true_b that also perturbs the
# axis.
N = 50
@vmap
def QuatNoise_perturbaxis_sample(key: int):
    return QuatNoise.addnoise_perturbaxis(key, b_true, sigma_dtheta, sigma_dphi)

# Split key again and get sample of observed bs with perturbed axis.
key, subkey = split(key)
bs_obs2 = QuatNoise_perturbaxis_sample(split(subkey, N))

# Notice that the third and fourth quaternion components are now nonzero!
print(bs_obs2[0])

[[ 0.06531329]
 [ 0.99785763]
 [-0.00281803]
 [-0.00252437]]


### Heading Noise Model

There may be times when you have a set of simulated heading vectors $\{\hat{\mathbf{u}}\}$ output from some ficticious sensor, like a star tracker or horizon sensor.  Heading vectors are represented as unit vectors since they only provide direction information.  This unit constraint means heading vectors are actually not really proper vectors since they lack distance information: adding two heading vectors $\hat{\mathbf{u}}_1 + \hat{\mathbf{u}}_2$ is not guaranteed to yield another valid heading vector.  As such, we need to be careful when injecting heading noise into our simulated 'true' heading vectors.

JAXitude adds noise to a 'true' heading vector $\hat{\mathbf{u}}$ using a rotation matrix $[\mathbf{R}(\delta\phi)]$, where $\delta\phi \sim \mathcal{N}(0, \sigma_{\delta\phi}$). 
The principal axis of this rotation matrix $[\mathbf{R}(\delta\phi)]$ is a random unit vector perpendicular to \hat{\mathbf{u}}.  To get a measured heading vector $\hat{\mathbf{u}}^*$, $[\mathbf{R}(\delta\phi)]$ is applied to the 'true' input unit vector $\mathbf{u}$:

$\mathbf{u}^* = [\mathbf{R}(\delta\phi)]\mathbf{u}$.

Let's use JAXitdues `jaxitude.operations.noise.HeadingNoise` class to simulate heading error.

In [12]:
from jaxitude.operations.noise import HeadingNoise

# Initial heading vector points along the y-axis.
u_true = jnp.array(
    [[0.],
     [1.],
     [0.]]
)

# Heading rotation angle standard deviation will be set to five degrees.
sigma_dphi = 5. * jnp.pi / 180.  # Convert to radians for JAXitude!

# wrapper function to generate N measured u from b_true.
N = 50
@vmap
def HeadingNoise_sample(key: int) -> jnp.ndarray:
    """ Calculates a sample of 
    """
    return HeadingNoise.addnoise(key, u_true, sigma_dphi)

# Let's also save the 'true' headings for comparison later.
key, subkey = split(key)
us_obs = HeadingNoise_sample(split(subkey, N))

# Let's see a simulated heading measurement.
print(us_obs[0])

[[-0.06917807]
 [ 0.9972563 ]
 [ 0.0263506 ]]
